## Анализ тональности отзывов на фильмы

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import movie_reviews

In [2]:
print(movie_reviews.readme())

Sentiment Polarity Dataset Version 2.0
Bo Pang and Lillian Lee

http://www.cs.cornell.edu/people/pabo/movie-review-data/

Distributed with NLTK with permission from the authors.


Introduction

This README v2.0 (June, 2004) for the v2.0 polarity dataset comes from
the URL http://www.cs.cornell.edu/people/pabo/movie-review-data .


What's New -- June, 2004

This dataset represents an enhancement of the review corpus v1.0
described in README v1.1: it contains more reviews, and labels were
created with an improved rating-extraction system.


Citation Info 

This data was first used in Bo Pang and Lillian Lee,
``A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization 
Based on Minimum Cuts'',  Proceedings of the ACL, 2004.

@InProceedings{Pang+Lee:04a,
  author =       {Bo Pang and Lillian Lee},
  title =        {A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts},
  booktitle =    "Proceedings of the ACL",
  year =      

In [3]:
movie_reviews.fileids()[:5]

['neg/cv000_29416.txt',
 'neg/cv001_19502.txt',
 'neg/cv002_17424.txt',
 'neg/cv003_12683.txt',
 'neg/cv004_12641.txt']

In [4]:
print(movie_reviews.raw('neg/cv001_19502.txt'))

the happy bastard's quick movie review 
damn that y2k bug . 
it's got a head start in this movie starring jamie lee curtis and another baldwin brother ( william this time ) in a story regarding a crew of a tugboat that comes across a deserted russian tech ship that has a strangeness to it when they kick the power back on . 
little do they know the power within . . . 
going for the gore and bringing on a few action sequences here and there , virus still feels very empty , like a movie going for all flash and no substance . 
we don't know why the crew was really out in the middle of nowhere , we don't know the origin of what took over the ship ( just that a big pink flashy thing hit the mir ) , and , of course , we don't know why donald sutherland is stumbling around drunkenly throughout . 
here , it's just " hey , let's chase these people around with some robots " . 
the acting is below average , even from the likes of curtis . 
you're more likely to get a kick out of her work in hallow

In [5]:
movie_reviews.words('neg/cv001_19502.txt')

['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...]

## Неделя 1: Строим простые модели

### Инструкция по выполнению

В некоторых пунктах нужно получить ответ - число или строку, которые будет нужно набирать в текстовых файлах и прикреплять в ответах на вопросы. Десятичные дроби записывайте через точку.

1. Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.
2. Подсчитайте количество отзывов в выборке.
3. Подсчитайте долю класса 1 в выборке.
4. Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.
5. Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.
6. Аналогично accuracy, оцените качество по ROC AUC.
7. Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.

In [6]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [7]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [8]:
corpus = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [9]:
corpus = [' '.join(text) for text in corpus]

Часть 1. Количество отзывов во всей выборке

In [10]:
len(corpus)

2000

Часть 2. Доля позитивных отзывов в выборке

In [11]:
sum(labels) / len(labels)

0.5

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vectorizer = CountVectorizer()

In [14]:
matrix = vectorizer.fit_transform(corpus)

Часть 3. Количество признаков в CountVectorizer со стандартными настройками

In [15]:
matrix.get_shape()[1]

39659

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [17]:
pipeline = Pipeline(steps=[('vectorizer', CountVectorizer()), ('regression', LogisticRegression())])

Часть 4. Accuracy в кросс-валидации

In [18]:
cross_val_score(pipeline, corpus, labels).mean()

0.8360216503929078

Часть 5. ROC AUC в кросс-валидации

In [19]:
cross_val_score(pipeline, corpus, labels, scoring='roc_auc').mean()

0.9107764937833774

Часть 6. Запишите соответствующие двум самым важным для модели признакам слова из словаря (без запятых).

In [20]:
pipeline.fit(corpus, labels)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [21]:
for feature in sorted(list(zip(abs(pipeline.named_steps.regression.coef_[0]), 
                               pipeline.named_steps.vectorizer.get_feature_names())), reverse=True)[:5]:
    print(feature[1], end=' ')

bad unfortunately worst fun waste 

## Неделя 2: Настройка параметров

### Инструкции:

1. Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество ( .mean() ) и стандартное отклонение ( .std() ) по fold'ам для: а) pipeline из CountVectorizer() и LogisticRegression(), б) pipeline из TfidfVectorizer() и LogisticRegression(). В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б

2. Попробуйте задавать разные значения параметра min_df у CountVectorizer. Оцените качество вашего классификатора с min_df=10 и с min_df=50.

3. Попробуйте использовать разные классификаторы после CountVectorizer. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.

4. Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.

5. Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.

In [22]:
def text_classifier(vectorizer, classifier):
    return Pipeline([('vectorizer', vectorizer), 
                     ('classifier', classifier)])

In [26]:
score_a = cross_val_score(text_classifier(CountVectorizer(), LogisticRegression()), corpus, labels, cv=5)
print(score_a.mean(), score_a.std())

0.8415000000000001 0.01677796173556255


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

Вопрос 1: В﻿ыпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б. Ответ округлить до 3 знаков после запятой.

In [30]:
score_b = cross_val_score(text_classifier(TfidfVectorizer(), LogisticRegression()), corpus, labels, cv=5)
print(score_a.mean(), score_a.std(), score_b.mean(), score_b.std())

0.8415000000000001 0.01677796173556255 0.8210000000000001 0.004062019202317978


Вопрос 2: Выпишите качество работы вашего классикатора с min_df=10 и с min_df=50 в CountVectorizer.

In [32]:
cross_val_score(text_classifier(CountVectorizer(min_df=10), LogisticRegression()), corpus, labels, cv=5).mean()

0.8390000000000001

In [33]:
cross_val_score(text_classifier(CountVectorizer(min_df=50), LogisticRegression()), corpus, labels, cv=5).mean()

0.813

Вопрос 3: Качество работы худшего классификатора

In [37]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

In [42]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    print(cross_val_score(text_classifier(CountVectorizer(), clf()), corpus, labels).mean())

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.8360216503929078
<class 'sklearn.svm.classes.LinearSVC'>
0.827517637397877
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>


C:\Users\d.v.kulemin.UTMN\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\d.v.kulemin.UTMN\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and de

0.7511493529457601


Вопрос 4: выпишете через пробел качество алгоритма со стоп-словами из nltk и из sklearn

In [48]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\d.v.kulemin.UTMN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [49]:
stop_list = stopwords.words('english')

In [50]:
stop_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [51]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    print(cross_val_score(text_classifier(CountVectorizer(stop_words=stop_list), clf()), corpus, labels).mean())

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.8270171368973763
<class 'sklearn.svm.classes.LinearSVC'>
0.817510624396852
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>


C:\Users\d.v.kulemin.UTMN\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\d.v.kulemin.UTMN\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and de

0.8065475655296015


In [52]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    print(cross_val_score(text_classifier(CountVectorizer(stop_words='english'), clf()), corpus, labels).mean())

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.8345141548734363
<class 'sklearn.svm.classes.LinearSVC'>
0.822010633387879
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>


C:\Users\d.v.kulemin.UTMN\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\d.v.kulemin.UTMN\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and de

0.8050100999202795


Вопрос 5: качество работы алгоритма на 1-2-граммах из слов и 3-5-граммах из букв

In [55]:
cross_val_score(text_classifier(CountVectorizer(ngram_range=(1,2)), LogisticRegression()), corpus, labels, cv=5).mean()

0.852

In [54]:
cross_val_score(text_classifier(CountVectorizer(ngram_range=(3,5), analyzer='char_wb'), LogisticRegression()), corpus, labels, cv=5).mean()

0.8205